In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
def chat(message, history):
    return f"You said {message} and the history is {history} but I still say bananas"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [4]:
load_dotenv(override=True)

True

In [9]:
groq_api_key = os.getenv('GROQ_API_KEY')
groq_url = "https://api.groq.com/openai/v1"
groq_api_key = os.getenv('GROQ_API_KEY')
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

Groq API Key exists and begins gsk_


In [7]:
ollama = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama" # We don't actually need a key for local, but the library asks for one
)

In [13]:
system_message = """
You are a helpful and enthusiastic assistant for a premium bus service called 'RoadRunner'.
Give short, courteous answers, no more than 1 sentence.
Always be accurate, but after one to two responses encourage the user to book a seat.
"""

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model="openai/gpt-oss-120b", messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

In [14]:
# A dictionary of specific routes. 
# Key = (City A, City B), Value = {Distance in KM, Base Price in PKR}
ROUTE_DB = {
    # Punjab Routes
    ("sialkot", "lahore"): {"km": 130, "price": 900},
    ("lahore", "islamabad"): {"km": 375, "price": 1800},
    ("sialkot", "islamabad"): {"km": 200, "price": 1200},
    ("faisalabad", "lahore"): {"km": 180, "price": 1100},
    ("multan", "lahore"): {"km": 340, "price": 1700},
    ("gujranwala", "lahore"): {"km": 80, "price": 600},
    ("sargodha", "lahore"): {"km": 190, "price": 1100},
    ("bahawalpur", "multan"): {"km": 100, "price": 800},
    
    # Long Haul (Motorway/Highway)
    ("karachi", "lahore"): {"km": 1200, "price": 6500},
    ("karachi", "hyderabad"): {"km": 160, "price": 1000},
    ("peshawar", "islamabad"): {"km": 180, "price": 1000},
    ("abbottabad", "islamabad"): {"km": 120, "price": 900},
    ("quetta", "karachi"): {"km": 690, "price": 4000},
    ("sukkur", "karachi"): {"km": 480, "price": 3000},
    ("murree", "islamabad"): {"km": 60, "price": 700}
}

# Average bus speed in Pakistan (Motorway vs GT Road average)
BUS_SPEED_KMH = 80

In [ ]:
def get_bus_details(source, destination):
    """
    Fetch price, distance, and estimated time for a bus trip.
    """
    # 1. Normalize inputs (Lowercase to match our DB)
    source = source.lower().strip()
    destination = destination.lower().strip()
    
    # 2. Try to find the route (Check both A->B and B->A)
    route_data = ROUTE_DB.get((source, destination))
    
    if not route_data:
        # Check reverse direction
        route_data = ROUTE_DB.get((destination, source))
        
    # 3. If still no route, return an error
    if not route_data:
        return f"Sorry, we currently do not have a direct route defined between {source.title()} and {destination.title()}."

    # 4. Extract Data
    distance = route_data["km"]
    price = route_data["price"]
    
    # 5. Calculate Time (Distance / Speed)
    # logic: Distance divided by Speed = Hours
    hours = distance / BUS_SPEED_KMH
    
    # Formatting time neatly (e.g., 2.5 hours -> 2h 30m)
    hours_int = int(hours)
    minutes_int = int((hours - hours_int) * 60)
    time_str = f"{hours_int}h {minutes_int}m"

    # 6. Return structured data (JSON style for the LLM)
    return {
        "source": source.title(),
        "destination": destination.title(),
        "price_pkr": price,
        "distance_km": distance,
        "estimated_time": time_str,
    }

In [16]:
# Test Case 1: Direct Match
print(get_bus_details("Sialkot", "Lahore"))
# Output: {'source': 'Sialkot', 'destination': 'Lahore', 'price_pkr': 900, 'distance_km': 130, 'estimated_time': '1h 37m', ...}

# # Test Case 2: Reverse Match (Logic Check)
# print(get_bus_details("Islamabad", "Lahore")) 
# # Output: (Same data as Lahore->Islamabad)

# # Test Case 3: Missing Route
# print(get_bus_details("Sialkot", "Karachi"))
# # Output: Sorry, we currently do not have a direct route...

{'source': 'Sialkot', 'destination': 'Lahore', 'price_pkr': 900, 'distance_km': 130, 'estimated_time': '1h 37m', 'note': 'Prices may vary due to fuel rates.'}


In [17]:
import json

# The Tool Schema
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_bus_details",
            "description": "Get price, distance, and time for a bus trip between two cities.",
            "parameters": {
                "type": "object",
                "properties": {
                    "source": {"type": "string", "description": "Starting city (e.g. Lahore)"},
                    "destination": {"type": "string", "description": "Destination city (e.g. Islamabad)"}
                },
                "required": ["source", "destination"]
            }
        }
    }
]

# The Handler Function (Runs your Python logic)
def handle_tool_calls(message):
    results = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_bus_details":
            # A. Parse the arguments from the LLM
            args = json.loads(tool_call.function.arguments)
            
            # B. Call the function we created earlier
            output = get_bus_details(args["source"], args["destination"])
            
            # C. Format the result back for the LLM
            results.append({
                "role": "tool", 
                "tool_call_id": tool_call.id, 
                "content": str(output)
            })
    return results

In [42]:
def chat(message, history):
    # 1. Convert Gradio history (list of dicts) to OpenAI format
    messages = [{"role": "system", "content": system_message}]
    for msg in history:
        messages.append({"role": msg["role"], "content": msg["content"]})
    
    # Add current user message
    messages.append({"role": "user", "content": message})

    # 2. First API Call (Ask LLM what to do)
    response = groq.chat.completions.create(
        model="openai/gpt-oss-120b",
        #model="qwen2.5:3b",
        messages=messages,
        tools=tools
    )

    # 3. Tool Loop
    while response.choices[0].finish_reason == "tool_calls":
        ai_message = response.choices[0].message

        # --- ADDED DEBUG PRINT HERE ---
        # This checks the first tool call in the list
        tool_name = ai_message.tool_calls[0].function.name
        tool_args = ai_message.tool_calls[0].function.arguments
        print(f"🛠️ AI is calling tool: {tool_name}")
        print(f"   └── Arguments: {tool_args}")
        # -----------------------------

        
        messages.append(ai_message)
        tool_results = handle_tool_calls(ai_message)
        messages.extend(tool_results)
        response = groq.chat.completions.create(
            model="openai/gpt-oss-120b",
            #model="qwen2.5:3b",
            messages=messages,
            tools=tools
        )

    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(
    fn=chat,
    title="RoadRunner Bus AI 🚌",
    description="Ask for prices between Sialkot, Lahore, Islamabad, or Karachi.",
    examples=["Price from Sialkot to Lahore?", "How far is Karachi from Lahore?"],
    type="messages" # Updated for newer Gradio versions
).launch()


In [43]:
import sqlite3

DB_NAME = "bus_service.db"

def setup_database():
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        
        # 1. Create a table for ROUTES (Source -> Destination)
        # We use a unique constraint so we don't duplicate routes
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS routes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                source TEXT NOT NULL,
                destination TEXT NOT NULL,
                price REAL,
                distance REAL,
                UNIQUE(source, destination)
            )
        ''')
        
        # 2. The Data (15+ Realistic Routes for Pakistan)
        # Format: (Source, Destination, Price, Distance)
        route_data = [
            ("sialkot", "lahore", 900, 130),
            ("lahore", "islamabad", 1800, 375),
            ("sialkot", "islamabad", 1200, 200),
            ("faisalabad", "lahore", 1100, 180),
            ("multan", "lahore", 1700, 340),
            ("gujranwala", "lahore", 600, 80),
            ("sargodha", "lahore", 1100, 190),
            ("bahawalpur", "multan", 800, 100),
            ("karachi", "lahore", 6500, 1200),
            ("karachi", "hyderabad", 1000, 160),
            ("peshawar", "islamabad", 1000, 180),
            ("abbottabad", "islamabad", 900, 120),
            ("quetta", "karachi", 4000, 690),
            ("sukkur", "karachi", 3000, 480),
            ("murree", "islamabad", 700, 60),
            ("basantpur", "peshawar", 950, 170)
        ]
        
        # 3. Insert Data (Using 'OR IGNORE' to avoid errors if you run this twice)
        cursor.executemany('''
            INSERT OR IGNORE INTO routes (source, destination, price, distance) 
            VALUES (?, ?, ?, ?)
        ''', route_data)
        
        conn.commit()
        print("✅ Database created and 16 routes inserted successfully.")

# Run the setup
setup_database()

✅ Database created and 16 routes inserted successfully.


In [44]:
import json

BUS_SPEED_KMH = 80

def get_bus_details(source, destination):
    print(f"🗄️ DATABASE QUERY: Checking route {source} <-> {destination}...", flush=True)
    
    source = source.lower().strip()
    destination = destination.lower().strip()
    
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        
        # SQL Query: Check if the route exists in EITHER direction
        query = '''
            SELECT price, distance FROM routes 
            WHERE (source=? AND destination=?) 
            OR (source=? AND destination=?)
        '''
        cursor.execute(query, (source, destination, destination, source))
        result = cursor.fetchone()
        
        if result:
            price, distance = result
            
            # Calculate Time
            hours = distance / BUS_SPEED_KMH
            time_str = f"{int(hours)}h {int((hours - int(hours))*60)}m"
            
            return json.dumps({
                "source": source.title(),
                "destination": destination.title(),
                "price": price,
                "distance": f"{distance} km",
                "estimated_time": time_str
            })
        else:
            return json.dumps({"error": f"No direct route found between {source} and {destination}."})

In [45]:
gr.ChatInterface(
    fn=chat,
    title="RoadRunner Bus AI 🚌",
    description="Ask for prices between Sialkot, Lahore, Islamabad, or Karachi.",
    examples=["Price from Sialkot to Lahore?", "How far is Karachi from Lahore?"],
    type="messages" # Updated for newer Gradio versions
).launch()


* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


🛠️ AI is calling tool: get_bus_details
   └── Arguments: {"destination":"Peshawar","source":"Basantpur"}
🗄️ DATABASE QUERY: Checking route Basantpur <-> Peshawar...
